# RAG from Scratch: Part 11 - Query Structuring

Resources:

- Video: [RAG from Scratch: Part 11](https://www.youtube.com/watch?v=kl6NwWYxvbM&list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x&index=11)
- Notebook: [`rag_from_scratch_10_and_11.ipynb`](./notebooks/rag-from-scratch/rag_from_scratch_10_and_11.ipynb)

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv(override=True, dotenv_path="../.env")

True